In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [15]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [16]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312504  [   64/60000]
loss: 2.303255  [ 6464/60000]
loss: 2.281967  [12864/60000]
loss: 2.274325  [19264/60000]
loss: 2.257807  [25664/60000]
loss: 2.224441  [32064/60000]
loss: 2.242484  [38464/60000]
loss: 2.202636  [44864/60000]
loss: 2.197214  [51264/60000]
loss: 2.179278  [57664/60000]
Test Error: 
 Accuracy: 38.0%, Avg loss: 2.169219 

Epoch 2
-------------------------------
loss: 2.177844  [   64/60000]
loss: 2.176737  [ 6464/60000]
loss: 2.115532  [12864/60000]
loss: 2.136334  [19264/60000]
loss: 2.097222  [25664/60000]
loss: 2.028299  [32064/60000]
loss: 2.072736  [38464/60000]
loss: 1.990304  [44864/60000]
loss: 1.992310  [51264/60000]
loss: 1.944982  [57664/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.929706 

Epoch 3
-------------------------------
loss: 1.958601  [   64/60000]
loss: 1.935447  [ 6464/60000]
loss: 1.814771  [12864/60000]
loss: 1.860901  [19264/60000]
loss: 1.767141  [25664/60000]
loss: 1.703900  [32064/600